In [1]:
import pandas as pd
import numpy as np
from pqd_dataset import PQDDataset
from sklearn.utils.class_weight import compute_class_weight
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.optim import Adam
from tqdm.auto import tqdm, trange
from pqd_trainer import train_model, train_ae_model, evaluate_model, evaluate_ae_model
from models.conv_mlp import CONV_MLP
from models.conv_ae_mlp import CONV_AE_MLP
from sklearn.metrics import f1_score, balanced_accuracy_score
import random
import os

In [2]:
window_size = 32
stride = 100
target_mode = True
epochs=100
batch_size=32
lr = 0.0005

In [3]:
test_df = pd.read_csv('test.csv')
test_dataset = PQDDataset(
                            df=test_df,
                            window_size=window_size,
                            stride=stride,
                            target_mode=target_mode
                            )

Creating window slices:   0%|          | 0/13 [00:00<?, ?it/s]

In [17]:
class Encoder(nn.Module):
    def __init__(self, autoencoder):
        super(Encoder, self).__init__()
        self.encoder = autoencoder.conv
        self.fc = autoencoder.enc_fc
        # Берем все слои энкодера до bottleneck

    def forward(self, x):
        x = self.encoder(x.permute(0, 2, 1))
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [18]:
dataloader = DataLoader(test_dataset,batch_size=10)

In [19]:
pretrained_autoencoder = torch.load("cnnae_model_seed206_ep93_tl0.9335.pt")
model = Encoder(pretrained_autoencoder)

In [21]:
dataset_with_latent = test_dataset.df.copy()
names_of_latent = []
for i in range(40):
    dataset_with_latent["l" + str(i)] = None
    names_of_latent.append("l" + str(i))
with torch.no_grad():
    for sample, target in tqdm(dataloader, desc="Step ...", leave=False):
        sample = sample
        output = model(sample).numpy()

        for j in range(batch.shape[0]):
            # пишу в первый семпл фрейма а не последний:
            id_of_batch_start = idx[:, 0][j]
            dataset_with_latent.loc[id_of_batch_start, names_of_latent] = (
                output[j, :]
            )
        print(
            "Line", id_of_batch_start, "from", len(dataset_with_latent) - 32
        )

dataset_with_latent.to_csv("cluster10.csv")
pass

Step ...:   0%|          | 0/18 [00:00<?, ?it/s]

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed